In [2]:
import pandas as pd
import ipywidgets as widgets
import numpy as np
import pathlib
import matplotlib.pyplot as plt
import datetime

In [3]:
PATH = pathlib.Path("./data")
data = pd.read_csv(PATH / "prepared_data.csv")
data["Date"] = pd.to_datetime(data["Date"])
data.index = data["Date"]
data.index.name = "DateIndex"

In [4]:
((data["Date"] + pd.DateOffset(1) - data["Date"]) == pd.Timedelta("1 days")).all()

True

In [20]:
(pd.to_datetime("2020-05-18") - pd.to_datetime("2020-05-20")).total_seconds() > 0

False

In [5]:
(data["Date"].dt.to_period('M').unique().to_timestamp())

<DatetimeArray>
['2016-06-01 00:00:00', '2016-07-01 00:00:00', '2016-08-01 00:00:00',
 '2016-09-01 00:00:00', '2016-10-01 00:00:00', '2016-11-01 00:00:00',
 '2016-12-01 00:00:00', '2017-01-01 00:00:00', '2017-02-01 00:00:00',
 '2017-03-01 00:00:00', '2017-04-01 00:00:00', '2017-05-01 00:00:00',
 '2017-06-01 00:00:00', '2017-07-01 00:00:00', '2017-08-01 00:00:00',
 '2017-09-01 00:00:00', '2017-10-01 00:00:00', '2017-11-01 00:00:00',
 '2017-12-01 00:00:00', '2018-01-01 00:00:00', '2018-02-01 00:00:00',
 '2018-03-01 00:00:00', '2018-04-01 00:00:00', '2018-05-01 00:00:00',
 '2018-06-01 00:00:00', '2018-07-01 00:00:00', '2018-08-01 00:00:00',
 '2018-09-01 00:00:00', '2018-10-01 00:00:00', '2018-11-01 00:00:00',
 '2018-12-01 00:00:00', '2019-01-01 00:00:00', '2019-02-01 00:00:00',
 '2019-03-01 00:00:00', '2019-04-01 00:00:00', '2019-05-01 00:00:00',
 '2019-06-01 00:00:00', '2019-07-01 00:00:00', '2019-08-01 00:00:00',
 '2019-09-01 00:00:00', '2019-10-01 00:00:00', '2019-11-01 00:00:00',
 '20

In [21]:
%matplotlib inline
%matplotlib widget
plt.close()
class App:
    def __init__(self, dataset):
        self.dataset = dataset
        dates = self.dataset["Date"].dt.to_period('M').unique().to_timestamp()
        options = [(i.strftime('%y-%b'), i) for i in dates]
        self.fig, self.ax = plt.subplots(1, 1)

        self.w_output = widgets.Output()

        self.w_date_from = widgets.DatePicker(description='From',
                                            disabled=False,
                                            value=dates[0],
                                            continuous_update=False)        
        self.w_date_to = widgets.DatePicker(description='To',
                                          disabled=False,
                                          value=dates[-1],
                                          continuous_update=False)
        
        self.w_description = widgets.Label(value="Select date range from to")
        self.w_information_status = widgets.Label(value="")
        
        self.w_date_slider = widgets.SelectionRangeSlider(
            options=options,
            description='2016-2020',
            disabled=False,
            continuous_update=False,
            value=(dates[0], dates[-1])
        )


        self.w_run_button = widgets.Button(description='Plot graph',disabled=False)
        self.w_run_button.on_click(self.on_button_clicked)

        self.w_box = widgets.HBox([self.w_date_from, self.w_date_to])
        self.wid = widgets.VBox([self.w_description, self.w_box, self.w_run_button, self.w_date_slider, self.w_information_status, self.w_output])

        self.w_date_from.observe(self.on_date_from_changed, "value")
        self.w_date_to.observe(self.on_date_to_changed, "value")
        self.w_date_slider.observe(self.on_date_slider_changed, "value")

    def check_valid_date_range(func):
        def wrapper(**kwargs):
            is_time_delta_positive = (date_to - date_from).total_seconds() > 0
            if is_time_delta_positive:
                self.w_information_status.value = ""
                func(**kwargs)
            else:
                with self.w_output:
                    self.w_information_status.value = "Invalid date range"
        return wrapper

    @check_valid_date_range
    def on_date_from_changed(self, value):
        date_range = slice(value["new"], self.w_date_to.value)
        self.replot_graph(date_range)

    def on_date_to_changed(self, value):
        date_range = slice(self.w_date_from.value, value["new"])
        self.replot_graph(date_range)

    def on_date_slider_changed(self, value):
        date_range = slice(value["new"][0], value["new"][-1])
        self.replot_graph(date_range)

    def on_button_clicked(self, button):
        date_range = slice(self.w_date_slider.value[0], self.w_date_slider.value[-1])
        self.replot_graph(date_range)

    def replot_graph(self, date_slice, **kwargs):
        with self.w_output:
            self.ax.cla()
            self.dataset.loc[date_slice].plot(y="Temp_avg",
                                              ylim=(10, 100),
                                              ax=self.ax,
                                              **kwargs)

    def run(self):
        return self.wid

app = App(data)
app.run()

NameError: name 'self' is not defined